In [ ]:
! pip install pandas transformers datasets scikit-learn simpletransformers torch

In [21]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [5]:
import pandas as pd

train_stances = pd.read_csv('train_stances.csv')
train_bodies = pd.read_csv('train_bodies.csv')
test_stances = pd.read_csv('competition_test_stances.csv')
test_bodies = pd.read_csv('competition_test_bodies.csv')

In [29]:
# Decide on a model from this list
model_name = "google/pegasus-xsum"
model_name = "google/pegasus-multi_news"

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [11]:
train_bodies.head(1)["articleBody"]

0    A small meteorite crashed into a wooded area i...
Name: articleBody, dtype: object

In [27]:
def summarize(src_text):
    batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text[0]

In [12]:
def summarize(text):
    tokens = tokenizer_model(text, truncation=True, padding="longest", return_tensors="pt")
    summary = loaded_model.generate(**tokens)
    return tokenizer_model.decode(summary[0])

In [24]:
train_bodies.head(1)["articleBody"][0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [28]:
summarize(str(train_bodies.head(1)["articleBody"]))

'A small meteorite crashed into a wooded area in the north-east of England, causing some residents to flee their homes.'